In [1]:
import numpy as np

import sys
sys.path.insert(0, '../../AgentTorch/')
from AgentTorch import Configurator

## Create Configurator

In [2]:
conf = Configurator()

## Add Metadata

In [3]:
conf.add_metadata('num_episodes', 10)
conf.add_metadata('num_steps_per_episode', 25)
conf.add_metadata('num_substeps_per_step', 1)
conf.add_metadata('device', 'cpu')
conf.add_metadata('num_agents', 6400)
conf.add_metadata('num_products', 2)

## Create Agents

In [4]:
num_agents = conf.get('simulation_metadata.num_agents')
conf.add_agents(key="consumers", number=6400)
conf.add_agents(key="marketers", number=2)

## Add Agent Properties

In [5]:
# ft property
from AgentTorch.helpers import random_normal

ft_args_list = [conf.create_variable(key='mu', name='ft_mean', shape=(1,), value=0.5, dtype="float"),
                conf.create_variable(key='sigma', name='ft_sigma', shape=(1,), value=0.67, dtype="float"),]

ft_initializer = conf.create_initializer(generator=random_normal, arguments=ft_args_list)

conf.add_property(root='state.agents.consumers', key='F_t', name='Follower Tendency', shape=(num_agents, 1), learnable=True, initialization_function=ft_initializer, dtype="float")


In [6]:
# purchased_before property
num_products = conf.get('num_products')
conf.add_property(root='state.agents.consumers', key='Purchased_before', name='Purhcased before', shape=(num_agents, num_products), value=0.5, initialization_function=None, dtype="float")

In [7]:
# q_exp
from models.opinion.substeps.utils import random_normal_col_by_col

qexp_args_list = [conf.create_variable(key='mu_0', name='Q_exp_mu_0', shape=(1,), value=0.5, initialization_function=None, dtype="float"),
                  conf.create_variable(key='mu_1', name='Q_exp_mu_1', shape=(1,), value=0.5, initialization_function=None, dtype="float"),
                  conf.create_variable(key='sigma_0', name='Q_exp_sigma_0', shape=(1,), value=0.2, initialization_function=None, dtype="float"),
                  conf.create_variable(key='sigma_1', name='Q_exp_sigma_1', shape=(1,), value=0.2, initialization_function=None, dtype="float")]

qexp_initializer = conf.create_initializer(generator=random_normal_col_by_col, arguments=qexp_args_list)
conf.add_property(root='state.agents.consumers', key='Q_exp', name='Quality Experienced', shape=(num_agents, num_products), initialization_function=qexp_initializer, dtype="float")


In [8]:
# q_des
conf.add_property(root='state.agents.consumers', key='Q_des', name='Quality desired', shape=(num_agents, 1), value=0.5, initialization_function=None, dtype="float")

## Add Objects

In [9]:
# add object 1

conf.add_objects(key='ProductA', number=1)

conf.add_property(root='state.objects.ProductA', key="cost", name="ProductA cost", shape=(1,1), initialization_function=False, value=1.0, dtype="float")
conf.add_property(root='state.objects.ProductA', key="quality", name="ProductA quality", shape=(1,1), initialization_function=False, value=1.0, dtype="float")

In [10]:
# add object 2

conf.add_objects(key='ProductB', number=1)

conf.add_property(root='state.objects.ProductB', key="cost", name="ProductB cost", shape=(1,1), initialization_function=False, value=1.0, dtype="float")
conf.add_property(root='state.objects.ProductB', key="quality", name="ProductB quality", shape=(1,1), initialization_function=False, value=1.0, dtype="float")

## Add Environment Network

In [11]:
# network
from AgentTorch.helpers import grid_network
conf.add_network(network_name="opinion_network", network_type=grid_network, arguments={'shape': [80, 80]})

## Add Substeps

In [12]:
from models.opinion.substeps.purchase_product.observation import GetFromState, GetNeighborsSum, GetNeighborsSumReduced

get_from_state = conf.create_function(generator=GetFromState, arguments=None, 
                                      input_variables={'F_t': 'agents/consumers/F_t', 'Q_des': 'num_agents/consumers/Q_des', 'Q_exp': 'agents/consumers/Q_exp'},
                                      output_variables=['F_t', 'Q_des', 'Q_exp'], fn_type="observation")

get_neighbors_sum = conf.create_function(
    generator=GetNeighborsSum, fn_type="observation",
    input_variables={'adjancency_matrix': "network/agent_agent/opinion_network/adjacency_matrix",
                    'query_feature': "agents/consumers/purchased_before"},
    output_variables=['N_i'],
    arguments=[conf.create_variable(key="neighborhood", name="Neighborhood Scope", shape=(1,), value=1, dtype="int", initialization_function=None)]
)

get_neighbors_sum_reduced = conf.create_function(
    generator=GetNeighborsSumReduced, fn_type="observation",
    input_variables={'adjancency_matrix': "network/agent_agent/opinion_network/adjacency_matrix",
                    'query_feature': "agents/consumers/purchased_before"},
    output_variables=['N_p'],
    arguments=[conf.create_variable(key="neighborhood", name="Neighborhood Scope", shape=(1,), value=1, dtype="int", initialization_function=None)]
)

/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: /Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so
  Reason: image not found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  

In [13]:
from models.opinion.substeps.purchase_product.action import PurchaseProduct
purchase_product_policy = conf.create_function(generator=PurchaseProduct, arguments=None, fn_type="policy",
                                               input_variables={'N_i': 'N_i', 'N_p': 'N_p', 'F_t': 'F_t', 'Q_des': 'Q_des', 'Q_exp': 'Q_exp'},
                                                output_variables=["purchase_action"])

In [14]:
from models.opinion.substeps.purchase_product.transition import NewQExp, NewPurchasedBefore

transition_new_q_exp = conf.create_function(generator=NewQExp, input_variables={'Q_exp': "agents/consumers/Q_exp"},
                                            output_variables=['Q_exp'], fn_type="transition",
                                            arguments=[
                                                conf.create_variable(key='distribution_params_0_mu', name="Resample dist mu 0", shape=[1,], value=0.5, initialization_function=None, dtype="float"),
                                                conf.create_variable(key='distribution_params_0_sigma', name="Resample dist sigma 0", shape=[1,], value=0.2, initialization_function=None, dtype="float"),
                                                conf.create_variable(key='distribution_params_1_mu', name="Resample dist mu 1", shape=[1,], value=0.5, initialization_function=None, dtype="float"),
                                                conf.create_variable(key='distribution_params_1_sigma', name="Resample dist sigma 1", shape=[1,], value=0.2, initialization_function=None, dtype="float")
                                            ])

transition_new_purchased_before = conf.create_function(generator=NewPurchasedBefore, arguments=None,
                                                        fn_type="transition",
                                                       input_variables={"Purchase_before": "agents/consumers/Purchased_before"},
                                                       output_variables=["Purchase_before"])

In [15]:
conf.add_substep(name="New Purchase", active_agents=['consumers'], 
                observation_fn=[get_from_state, get_neighbors_sum, get_neighbors_sum_reduced],
                policy_fn = [purchase_product_policy],
                transition_fn = [transition_new_q_exp, transition_new_purchased_before])

## Render simulator

In [16]:
conf.render('opdyn_config.yaml')